In [1]:
import pandas as pd
import numpy as np
import folium
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import datetime
import requests
import geopandas as gpd
from datetime import timedelta

In [2]:
df = pd.read_csv('Locatievoorstellen.csv', header=1, index_col=0)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.columns = df.columns.str.replace(' - ', '_')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()

In [5]:
df["tijdstip_van_laatste_wijziging"] = pd.to_datetime(df["tijdstip_van_laatste_wijziging"])
df["tijdstip_aangemaakt"] = pd.to_datetime(df["tijdstip_aangemaakt"])
df["tijdstip_locatievoorstel"] = pd.to_datetime(df["tijdstip_locatievoorstel"])
df["tijdstip_controle_locatievoorstel"] = pd.to_datetime(df["tijdstip_controle_locatievoorstel"])
df["tijdstip_advies_exploitant"] = pd.to_datetime(df["tijdstip_advies_exploitant"])
df["tijdstip_publicatie_verkeersbesluit"] = pd.to_datetime(df["tijdstip_publicatie_verkeersbesluit"])
df["tijdstip_aanvraag_netaansluiting"] = pd.to_datetime(df["tijdstip_aanvraag_netaansluiting"])
df["tijdstip_definitief_verkeersbesluit"] = pd.to_datetime(df["tijdstip_definitief_verkeersbesluit"])
df["tijdstip_accepteren_mra-e"] = pd.to_datetime(df["tijdstip_accepteren_mra-e"])
df["tijdstip_realisatie_plannen"] = pd.to_datetime(df["tijdstip_realisatie_plannen"])
df["tijdstip_locatie_voorbereiden"] = pd.to_datetime(df["tijdstip_locatie_voorbereiden"])
df["tijdstip_in_bedrijf"] = pd.to_datetime(df["tijdstip_in_bedrijf"])
df["tijdstip_opleveren_laadpaal"] = pd.to_datetime(df["tijdstip_opleveren_laadpaal"])
df["tijdstip_opleveren_locatie"] = pd.to_datetime(df["tijdstip_opleveren_locatie"])

In [6]:
df['doorlooptijd_d'] = (df['tijdstip_opleveren_locatie'] - df['tijdstip_aangemaakt'])

In [7]:
df = df.drop(columns=['stilgezet_uitleg','verplaatsing_mra-e_laadpaal_id', 'opmerkingen', 'toelichting_bij_verplaatsing', 
                      'locatiecode', 'aanbodgestuurd_opmerkingen', 'datagestuurd_opmerkingen', 'gemeente_akkoord_met_offerte_verplaatsing',
                     'extra_laadpaal_ids', 'referenties', 'aanleiding'])

In [8]:
df['postcode'] = df['postcode'].str.replace('[a-zA-Z]', '', regex=True)
df['postcode'] = df['postcode'].str.replace(' ', '', regex=True)

In [9]:
df.loc[(df['stilgezet'] == False) & df['doorlooptijd_d'].notna(), ['exploitant_akkoord', 'aannemer_akkoord', 'gemeente_akkoord']] = True
df.loc[(df['stilgezet'] == True) | df['doorlooptijd_d'].isna(), ['exploitant_akkoord', 'aannemer_akkoord', 'gemeente_akkoord']] = False

In [10]:
df['datum_oplevering_y'] = pd.DatetimeIndex(df['datum_oplevering']).year
df['datum_oplevering_y'] = df['datum_oplevering_y'].fillna(0).astype(int)
df.loc[(df['datum_oplevering_y'] == 0), 'datum_oplevering_y'] = None

In [11]:
df = df.loc[(df['datum_oplevering_y'] < 2024)]

Vanaf hier is het lokaal

In [12]:
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df['locatie_longitude'], df['locatie_latitude']))
gdf = gdf.set_crs('EPSG:4326')

In [13]:
gebieden = gpd.read_file('BestuurlijkeGebieden_2023.gml')
gebieden = gebieden.to_crs('EPSG:4326')
prov = gebieden.dissolve(by = 'ligtInProvincieNaam')
prov = prov.reset_index()[['ligtInProvincieNaam','geometry']]
gdf = gdf.sjoin(prov, predicate = 'within')

In [14]:
gdf.rename(columns = {'ligtInProvincieNaam':'provincie'}, inplace = True)

In [15]:
gdf.head()

,locatievoorstel_id,stilgezet,stilgezet_nu_of_ooit_in_verleden,tijdstip_van_laatste_wijziging,tijdstip_aangemaakt,tijdstip_locatievoorstel,tijdstip_controle_locatievoorstel,tijdstip_advies_exploitant,tijdstip_publicatie_verkeersbesluit,tijdstip_aanvraag_netaansluiting,tijdstip_definitief_verkeersbesluit,tijdstip_accepteren_mra-e,tijdstip_realisatie_plannen,tijdstip_locatie_voorbereiden,tijdstip_in_bedrijf,tijdstip_opleveren_laadpaal,tijdstip_opleveren_locatie,organisatie_id,organisatie_naam,gemeente_id,gemeente_naam,locatie_longitude,locatie_latitude,straat,huisnummer,postcode,plaats,was_voorkeurslocatie,voorkeurslocatieid,autodelen,laadplein,laadpunten,gereserveerde_parkeervakken,gereserveerde_parkeervakken_autodelen,exploitant_akkoord,aannemer_akkoord,gemeente_akkoord,verkeersbesluit_gepubliceerd_datum,verkeersbesluit_definitief_datum,verkeersbesluit_definitief_gewijzigd,aanvraag_netaansluiting_datum,datum_realisatie_gepland,datum_in_bedrijf,laadpaal_id,datum_oplevering,doorlooptijd_d,datum_oplevering_y,geometry,index_right,provincie
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,LV-6,False,False,2020-10-22 09:06:27.400,2018-12-05 13:40:44.360,2018-12-05 14:06:07.114,2018-12-19 14:45:00.395,2018-12-24 14:47:22.368,2018-12-31 08:50:02.188,2019-01-31 13:11:05.446,2018-12-31 10:46:40.870,2018-12-31 10:46:44.903,2019-04-24 05:54:56.583,2019-04-24 06:36:17.898,2019-05-08 08:36:20.072,2019-08-14 08:56:36.783,2019-09-11 07:24:17.680,84,OVER-Gemeenten,385,Wormerland,4.810485,52.494985,Spatterstraat,21,1531,Wormer,False,<nil>,False,False,2,2,0,True,True,True,2018-08-28 00:00:00,2018-10-09 00:00:00,False,2019-01-21 00:00:00,2019-05-06 00:00:00,2019-05-07 00:00:00,PP001911,2019-05-08 00:00:00,279 days 17:43:33.320000,2019.0,POINT (4.81049 52.49498),7,Noord-Holland
3,LV-11,False,False,2020-10-22 09:06:27.400,2018-12-07 09:59:17.463,2018-12-07 10:03:25.510,2019-01-14 12:10:51.520,2019-01-15 08:45:00.559,2019-01-31 15:28:01.963,2019-02-18 14:02:10.212,2019-02-20 07:31:18.342,2019-02-20 11:24:01.657,2019-06-07 06:57:55.205,2019-06-11 06:57:01.107,2019-06-13 12:08:19.299,2019-08-14 08:58:25.607,2020-07-16 14:50:16.003,28,Gemeente Heemskerk,334,Heemskerk,4.655286,52.511481,Commandeurslaan,81,1964,Heemskerk,False,<nil>,False,False,2,2,0,True,True,True,2019-01-25 00:00:00,2019-02-27 00:00:00,False,2019-02-14 00:00:00,2019-06-11 00:00:00,2019-06-13 00:00:00,PP002024,2019-06-13 00:00:00,587 days 04:50:58.540000,2019.0,POINT (4.65529 52.51148),7,Noord-Holland
4,LV-12,False,False,2020-10-22 09:06:27.400,2018-12-07 10:17:23.991,2018-12-07 10:18:44.540,2018-12-19 14:45:58.376,2018-12-24 14:49:50.718,2019-01-31 15:33:03.486,2019-02-18 14:04:13.754,2019-02-20 07:25:32.040,2019-02-20 11:24:13.567,2019-06-07 07:00:04.976,2019-06-11 06:59:11.801,2019-06-13 12:07:04.829,2019-08-14 09:00:13.784,2020-07-16 14:55:31.495,28,Gemeente Heemskerk,334,Heemskerk,4.670446,52.508710,Maerten van Heemskerckstraat,25,1961,Heemskerk,False,<nil>,False,False,2,2,0,True,True,True,2019-01-18 00:00:00,2019-02-20 00:00:00,False,2019-02-14 00:00:00,2019-06-12 00:00:00,2019-06-13 00:00:00,PP002023,2019-06-13 00:00:00,587 days 04:38:07.504000,2019.0,POINT (4.67045 52.50871),7,Noord-Holland
5,LV-13,False,False,2020-10-22 09:06:27.400,2018-12-11 12:38:02.088,2019-01-31 15:45:37.668,2019-02-25 12:05:58.888,2019-02-25 13:45:57.918,2019-03-19 10:36:27.679,2019-03-25 08:51:28.078,2019-04-30 07:15:35.345,2019-04-30 07:26:09.665,2019-09-11 15:02:04.703,2020-01-27 12:13:29.181,2019-09-30 10:32:23.344,2020-01-23 10:33:28.840,2020-01-27 12:13:32.395,24,Gemeente Enkhuizen,328,Enkhuizen,5.281177,52.701562,Nassaustraat,2,1601,Enkhuizen,False,<nil>,False,False,2,2,0,True,True,True,2019-03-19 00:00:00,2019-04-30 00:00:00,False,2019-03-22 00:00:00,2019-09-27 00:00:00,2019-09-27 00:00:00,PP002135,2019-09-27 00:00:00,411 days 23:35:30.307000,2019.0,POINT (5.28118 52.70156),7,Noord-Holland
11,LV-20,False,False,2020-10-22 09:06:27.400,2018-12-12 14:02:38.651,2018-12-12 14:07:23.320,2018-12-31 

In [16]:
doorlooptijd_df = gdf[['tijdstip_aanvraag_netaansluiting', 'tijdstip_locatie_voorbereiden', 
                       'tijdstip_in_bedrijf', 'tijdstip_realisatie_plannen', 'tijdstip_accepteren_mra-e',
                       'provincie', 'locatie_latitude', 'locatie_longitude']].copy()

In [17]:
doorlooptijd_df = doorlooptijd_df.loc[(doorlooptijd_df['tijdstip_locatie_voorbereiden'].notna())]

In [18]:
doorlooptijd_df['doorlooptijd_aanvraag_voorbereiding'] = (doorlooptijd_df['tijdstip_realisatie_plannen'] - doorlooptijd_df['tijdstip_aanvraag_netaansluiting'])
doorlooptijd_df['doorlooptijd_voorbereiding_overdracht'] = (doorlooptijd_df['tijdstip_locatie_voorbereiden'] - doorlooptijd_df['tijdstip_accepteren_mra-e'])
doorlooptijd_df['doorlooptijd_ontvangst_oplevering'] = (doorlooptijd_df['tijdstip_in_bedrijf'] - doorlooptijd_df['tijdstip_accepteren_mra-e'])
doorlooptijd_df['doorlooptijd_aanvraag_oplevering'] = (doorlooptijd_df['tijdstip_in_bedrijf'] - doorlooptijd_df['tijdstip_aanvraag_netaansluiting'])

In [19]:
doorlooptijd = [doorlooptijd_df['doorlooptijd_aanvraag_voorbereiding'], doorlooptijd_df['doorlooptijd_voorbereiding_overdracht'], 
             doorlooptijd_df['doorlooptijd_ontvangst_oplevering'], doorlooptijd_df['doorlooptijd_aanvraag_oplevering']]


In [20]:
# Convert to days
seconds_in_day = 86400
for i in range(len(doorlooptijd)):
    doorlooptijd[i] = doorlooptijd[i].dt.total_seconds() / seconds_in_day

In [21]:
doorlooptijd_in_dagen = pd.DataFrame(doorlooptijd).T
doorlooptijd_in_dagen.columns = ['aanvraag_voorbereiding', 'voorbereiding_overdracht', 'ontvangst_oplevering', 'aanvraag_oplevering']
doorlooptijd_in_dagen[['provincie', 'latitude', 'longitude']] = doorlooptijd_df[['provincie', 'locatie_latitude', 'locatie_longitude']]
doorlooptijd_in_dagen_geo = gpd.GeoDataFrame(doorlooptijd_in_dagen, geometry = gpd.points_from_xy(doorlooptijd_in_dagen['longitude'], doorlooptijd_in_dagen['latitude']))

In [22]:
doorlooptijd_in_dagen.loc[(doorlooptijd_in_dagen['aanvraag_voorbereiding'] == 0), 'aanvraag_voorbereiding'] = None
doorlooptijd_in_dagen.dropna(inplace = True)

In [23]:
# Drop rows that have 0 days in any of the columns
doorlooptijd_in_dagen = doorlooptijd_in_dagen.loc[(doorlooptijd_in_dagen['aanvraag_voorbereiding'] > 0)]

In [24]:
doorlooptijd_in_dagen.head()

,aanvraag_voorbereiding,voorbereiding_overdracht,ontvangst_oplevering,aanvraag_oplevering,provincie,latitude,longitude,geometry
0,,,,,,,,
1,82.697120,113.826076,127.909435,96.809197,Noord-Holland,52.494985,4.810485,POINT (4.81049 52.49498)
3,108.705382,110.814577,113.030760,114.920939,Noord-Holland,52.511481,4.655286,POINT (4.65529 52.51148)
4,108.705454,110.815952,113.029760,114.918647,Noord-Holland,52.508710,4.670446,POINT (4.67045 52.50871)
5,170.257368,272.199531,153.129325,189.070084,Noord-Holland,52.701562,5.281177,POINT (5.28118 52.70156)
11,87.221193,207.988673,115.067635,122.263100,Noord-Holland,52.521105,4.962389,POINT (4.96239 52.52110)


Plotting -------------------------------------------------------------

In [25]:
hist_data = [doorlooptijd_in_dagen['aanvraag_voorbereiding'], doorlooptijd_in_dagen['voorbereiding_overdracht'], 
             doorlooptijd_in_dagen['ontvangst_oplevering'], doorlooptijd_in_dagen['aanvraag_oplevering']]
labels = ['aanvraag - voorbereiding', 'voorbereiding - overdracht', 'ontvangst - oplevering', 'aanvraag - oplevering (totale doorlooptijd))']

fig = ff.create_distplot(hist_data=hist_data, group_labels=labels, bin_size=1, show_rug=False, show_hist=False, show_curve=True, colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
fig.update_layout(title='Doorlooptijd netbeheerder per aanvraag', xaxis_title='Tijd (dagen)', yaxis_title='Frequentie', 
                  legend_title='Aanvragen' ,font=dict(size=12))
fig.show()

In [28]:
fig = px.scatter_ternary(doorlooptijd_in_dagen, a="ontvangst_oplevering", b="voorbereiding_overdracht", c="aanvraag_voorbereiding", color='provincie', size='aanvraag_oplevering',
                         color_discrete_sequence=px.colors.qualitative.Dark24, size_max=15, opacity=0.8, hover_name='provincie',)
fig.show()

In [35]:
data = dict(
    number=[doorlooptijd_in_dagen['aanvraag_voorbereiding'].mean(), doorlooptijd_in_dagen['voorbereiding_overdracht'].mean(), 
            doorlooptijd_in_dagen['ontvangst_oplevering'].mean(), doorlooptijd_in_dagen['aanvraag_oplevering'].mean()],
    stage=['aanvraag - voorbereiding', 'voorbereiding - overdracht', 'ontvangst - oplevering', 'aanvraag - oplevering (totale doorlooptijd))']
)

fig = px.funnel(data, x='number', y='stage')
fig.show()